In [1]:




# ##Part B

# #Unweighted Page Rank
# unweighted_PR = nx.pagerank(G, alpha = 0.5, max_iter=100)

# #Weighted Page Rank
# ##Creating the network graph
# FG=nx.Graph()

# #Creating the labels for the nodes and adding a node to the G object
# labels = {}
# opp_labels = {}
# for i, value in enumerate(names):
#     labels[i] = str(value)
#     opp_labels[value] = i
#     FG.add_node(value) #Adding the nodes
# #Create the list of tuples for edges with weights this time
# node_list = []
# for index, pref in enumerate(averages_df.loc[:,'Positive']): #Positives
#     if ~np.isnan(pref):
#         node_list.append((pd.Series(averages_df.loc[:,'Node 1'])[index],pd.Series(averages_df.loc[:,'Node 2'])[index],pref))
# FG.add_weighted_edges_from(node_list)


# for index, pref in enumerate(averages_df.loc[:,'Negative']): #Negatives
#     if ~np.isnan(pref):
#         node_list.append((pd.Series(averages_df.loc[:,'Node 2'])[index],pd.Series(averages_df.loc[:,'Node 1'])[index],abs(pref)))
# FG.add_weighted_edges_from(node_list)


# weighted_PR = nx.pagerank(FG, alpha = 0.5, weight = 'weight')#, max_iter=100000, tol=1e-01)


# #Placing the PageRank in a series for combination and Correlation Calculation
# i=0
# unweighted_PageRank = pd.Series(name='Unweighted')
# for dictkey in unweighted_PR:
#     unweighted_PageRank[labels[i]] = unweighted_PR[dictkey]
#     i = i + 1

# weighted_PageRank = pd.Series(name='Weighted')
# for dictkey in weighted_PR:
#     weighted_PageRank[dictkey] = weighted_PR[dictkey]
    
# #Create a dict of price data
# price_dict = {'A6':20000, 'A8':12000, '3series':220000, '5series':60000, '7series':14000, 'XJ':6600, 'ES':135000, 'LS':30000, 'RX':120000, 'Sclass':25000}


# price_list = pd.Series(name='Price')
# for dictkey in price_dict:
#     price_list[dictkey] = price_dict[dictkey]

# price_pagerank = pd.concat([unweighted_PageRank, weighted_PageRank, price_list], axis=1)

# #np.correlate(price_pagerank['Unweighted'], price_pagerank['Price'])

# print('Unweighted Correlation to Price')
# print(scipy.stats.spearmanr(price_pagerank['Unweighted'], price_pagerank['Price']))
# print('Weighted Correlation to Price')
# print(scipy.stats.spearmanr(price_pagerank['Weighted'], price_pagerank['Price']))




In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy
import scipy.stats

rawsentiment = pd.read_csv(r'Assignment 3 Sentiment Scores (2).csv')


In [140]:

# #----------For Part C--------

rawsentiment = pd.read_csv(r'Assignment 3 Sentiment Scores (2).csv')
# rawsentiment = rawsentiment1.drop(['Unnamed: 0','model_strings'], axis=1)
#rawsentiment = rawsentiment1.drop('Unnamed: 0', axis=1) #Testing bejans
lab_for_C = ['ES', 'LS', 'RX', 'A8', 'A6', '3series', '5series', '7series', 'XJ', 'Sclass']
rawsentiment.columns = lab_for_C
rawsentiment = rawsentiment * 5
rawsentiment.replace(to_replace=0,value=0.0001, inplace=True)




In [141]:
rawsentiment.head()

,ES,LS,RX,A8,A6,3series,5series,7series,XJ,Sclass
0,NaN,NaN,20,15,NaN,NaN,NaN,25,10,5
1,NaN,NaN,15,10,10,10,10,10,NaN,NaN
2,NaN,20,NaN,NaN,NaN,15,NaN,15,NaN,15
3,NaN,10,NaN,10,NaN,NaN,NaN,10,20,10
4,NaN,15,15,NaN,NaN,NaN,10,NaN,NaN,15


In [142]:
#names = ['ES-LS','ES-RX','ES-A8','ES-A6','ES-3series','ES-5series','ES-7series','ES-XJ','ES-Sclass','LS-RX','LS-A8','LS-A6','LS-3series','LS-5series','LS-7series','LS-XJ','LS-Sclass','RX-A8','RX-A6','RX-3series','RX-5series','RX-7series','RX-XJ','RX-Sclass','A8-A6','A8-3series','A8-5series','A8-7series','A8-XJ','A8-Sclass','A6-3series','A6-5series','A6-7series','A6-XJ','A6-Sclass','3series-5series','3series-7series','3series-XJ','3series-Sclass','5series-7series','5series-XJ','5series-Sclass','7series-XJ','7series-Sclass','XJ-Sclass']
names = list(rawsentiment.columns)
#i=1
#j=1

##Create the sentiment difference dataframe (ES-LS, ES-RX, etc)
length_senti = len(rawsentiment)
width_senti = len(rawsentiment.T)
comparisons = pd.DataFrame(index=np.array(range(length_senti)))
#Vectors for future use in the nodes
node_name_1 = []
node_name_2 = []

n=1 #For counter
for i in range(width_senti-1):
    for j in range(width_senti - n):
        comparisons[str(names[i]) + '-' + str(names[(j+n)])] = np.array(rawsentiment[[i]]) - np.array(rawsentiment[[j+n]])
        node_name_1.append(names[i])
        node_name_2.append(names[(j+n)])
    n=n+1

length_comp = len(comparisons.T)

averages_df = pd.DataFrame(index=comparisons.columns, columns = ['Positive','Negative'])



In [143]:
averages_df.head()

,Positive,Negative
ES-LS,NaN,NaN
ES-RX,NaN,NaN
ES-A8,NaN,NaN
ES-A6,NaN,NaN
ES-3series,NaN,NaN


In [144]:



for i in comparisons.columns:
    pos_sum = 0
    neg_sum = 0    
    pos_sum = [pos_sum + x for x in comparisons[i] if x > 0]
    neg_sum = [neg_sum + x for x in comparisons[i] if x < 0]
    if len(pos_sum) != 0:    
        averages_df.loc[i,'Positive'] = sum(pos_sum)/len(pos_sum)
    if len(neg_sum) != 0:
        averages_df.loc[i,'Negative'] = sum(neg_sum)/len(neg_sum)

averages_df['Node 1'] = node_name_1
averages_df['Node 2'] = node_name_2


In [145]:
averages_df.head()

,Positive,Negative,Node 1,Node 2
ES-LS,5.833333,-10,ES,LS
ES-RX,8.333333,-10,ES,RX
ES-A8,17,-15,ES,A8
ES-A6,30,-8.333333,ES,A6
ES-3series,16.25,-30,ES,3series


In [146]:

##Creating the network graph
G=nx.DiGraph()

#Creating the labels for the nodes and adding a node to the G object
labels = {}
opp_labels = {}
for i, value in enumerate(names):
    labels[i] = str(value)
    opp_labels[value] = i
    G.add_node(i) #Adding the nodes
    

#Create the list of tuples for edges
node_list = []
for index, pref in enumerate(averages_df.loc[:,'Positive']): #Positives
    if ~np.isnan(pref):
        node_list.append((pd.Series(averages_df.loc[:,'Node 1'])[index],pd.Series(averages_df.loc[:,'Node 2'])[index]))
        G.add_edge(*(opp_labels[pd.Series(averages_df.loc[:,'Node 1'])[index]],opp_labels[pd.Series(averages_df.loc[:,'Node 2'])[index]]))
for index, pref in enumerate(averages_df.loc[:,'Negative']): #Negatives
    if ~np.isnan(pref):
        node_list.append((pd.Series(averages_df.loc[:,'Node 2'])[index],pd.Series(averages_df.loc[:,'Node 1'])[index]))
        G.add_edge(*(opp_labels[pd.Series(averages_df.loc[:,'Node 2'])[index]],opp_labels[pd.Series(averages_df.loc[:,'Node 1'])[index]]))


In [147]:
node_list

[('ES', 'LS'),
 ('ES', 'RX'),
 ('ES', 'A8'),
 ('ES', 'A6'),
 ('ES', '3series'),
 ('ES', '5series'),
 ('ES', '7series'),
 ('ES', 'XJ'),
 ('ES', 'Sclass'),
 ('LS', 'RX'),
 ('LS', 'A8'),
 ('LS', 'A6'),
 ('LS', '3series'),
 ('LS', '5series'),
 ('LS', '7series'),
 ('LS', 'XJ'),
 ('LS', 'Sclass'),
 ('RX', 'A8'),
 ('RX', 'A6'),
 ('RX', '3series'),
 ('RX', '5series'),
 ('RX', '7series'),
 ('RX', 'XJ'),
 ('RX', 'Sclass'),
 ('A8', 'A6'),
 ('A8', '3series'),
 ('A8', '5series'),
 ('A8', '7series'),
 ('A8', 'XJ'),
 ('A8', 'Sclass'),
 ('A6', '3series'),
 ('A6', '5series'),
 ('A6', 'XJ'),
 ('A6', 'Sclass'),
 ('3series', '7series'),
 ('3series', 'Sclass'),
 ('5series', '7series'),
 ('5series', 'Sclass'),
 ('7series', 'XJ'),
 ('7series', 'Sclass'),
 ('XJ', 'Sclass'),
 ('LS', 'ES'),
 ('RX', 'ES'),
 ('A8', 'ES'),
 ('A6', 'ES'),
 ('3series', 'ES'),
 ('5series', 'ES'),
 ('7series', 'ES'),
 ('XJ', 'ES'),
 ('Sclass', 'ES'),
 ('RX', 'LS'),
 ('A8', 'LS'),
 ('A6', 'LS'),
 ('3series', 'LS'),
 ('5series', 'LS'),


In [148]:
pos = nx.circular_layout(G) #Positions

#nx.edges([(1,2),(1,0)])#node_list)

nx.draw_networkx_labels(G,pos, labels, font_size=16)
#nx.draw_networkx_nodes(G,pos,node_color='skyblue')
nx.draw(G,pos,node_size=1000)
###nx.Graph(node_list)

fig1 = plt.gcf()
plt.figure(num=None, figsize=(30,20))
#plt.savefig('network.png', dpi=100)

plt.show()
fig1.savefig('network.png', dpi=1000)


In [149]:
averages_df.head(45)

,Positive,Negative,Node 1,Node 2
ES-LS,5.833333,-10,ES,LS
ES-RX,8.333333,-10,ES,RX
ES-A8,17,-15,ES,A8
ES-A6,30,-8.333333,ES,A6
ES-3series,16.25,-30,ES,3series
ES-5series,20,-25,ES,5series
ES-7series,18,-14,ES,7series
ES-XJ,5,-5,ES,XJ
ES-Sclass,16.42857,-13.07692,ES,Sclass
LS-RX,17,-5.714286,LS,RX


In [155]:
A = pd.DataFrame(index = names,columns = names)
A.ix['ES','ES']
A = A.fillna(0)
tuples  = averages_df
tuples.head()
tuples[:10]

length = len(tuples)
# write aggfunc
agg = pd.Series(index = names).fillna(0)
for i in range(length): 
    tup = tuples.ix[i]
    node1 = tup['Node 1']
    node2 = tup['Node 2']
    pos = abs(tup['Positive'])
    neg = abs(tup['Negative'])
    if ~np.isnan(pos):
        agg[node2] += pos #for weighted
#         agg[node2] += 1 # unweighted
    if ~np.isnan(neg):
        agg[node1] += neg
#         agg[node1] += 1 #for unweighted
    
agg['ES']

for i in range(length):
    tup = tuples.ix[i]
    node1 = tup['Node 1']
    node2 = tup['Node 2']
    pos = abs(tup['Positive'])
    neg = abs(tup['Negative'])
    if ~np.isnan(pos):
        A.ix[node1, node2] += pos/agg[node2]
    if ~np.isnan(neg):
        A.ix[node2, node1] += neg/agg[node1]
    

In [156]:
A

,ES,LS,RX,A8,A6,3series,5series,7series,XJ,Sclass
ES,0.000000,0.053396,0.122850,0.117936,0.189225,0.131935,0.184900,0.143357,0.068016,0.125951
LS,0.076681,0.000000,0.250614,0.099106,0.105125,0.113667,0.124807,0.105242,0.187045,0.097904
RX,0.076681,0.052306,0.000000,0.076312,0.115637,0.162382,0.166410,0.149330,0.136032,0.139392
A8,0.115022,0.129882,0.073710,0.000000,0.043364,0.131935,0.154083,0.086883,0.136032,0.095832
A6,0.063901,0.058844,0.000000,0.109842,0.000000,0.121786,0.138675,0.000000,0.068016,0.076666
3series,0.230043,0.183072,0.294840,0.173435,0.157687,0.000000,0.000000,0.199107,0.000000,0.166109
5series,0.191703,0.141880,0.000000,0.173435,0.157687,0.000000,0.000000,0.092916,0.000000,0.114999
7series,0.107354,0.125862,0.073710,0.104061,0.157687,0.081191,0.046225,0.000000,0.249393,0.119258
XJ,0.038341,0.132218,0.000000,0.074329,0.000000,0.121786,0.092450,0.126101,0.000000,0.063888
Sclass,0.100275,0.122540,0.184275,0.071542,0.073587,0.135318,0.092450,0.097064,0.155466,0.000000


In [152]:
# len(tuples)

In [153]:
# # write aggfunc
# agg = pd.Series(index = names).fillna(0)
# for i in range(20): 
#     tup = tuples.ix[i]
#     node1 = tup['Node 1']
#     node2 = tup['Node 2']
#     pos = abs(tup['Positive'])
#     neg = abs(tup['Negative'])
#     if ~np.isnan(pos):
#         agg[node2] += pos
#     if ~np.isnan(neg):
#         agg[node1] += neg
    
# agg['ES']

In [157]:
#convergence 
x = [1] * 10
from sklearn.metrics import mean_squared_error
from math import sqrt
tol = 10
while tol > 1e-7: 
    old = np.dot(A,x)
    tol = sqrt(mean_squared_error(old, x))
    x = old
    print tol
    
final_rankscore = x

0.221478444912
0.0408044027132
0.0116077126932
0.00375563399525
0.00122575160399
0.000375421700075
0.000105874808305
2.74361781561e-05
6.72338033178e-06
1.74610459048e-06
5.4726617152e-07
1.88836960266e-07
6.25342795115e-08


In [158]:
names = agg.index.values
ranks  = pd.Series(x, index = names)
ranks

ES         1.104954
LS         1.129851
RX         1.063558
A8         0.986219
A6         0.667003
3series    1.424546
5series    0.861935
7series    1.004711
XJ         0.711723
Sclass     1.045500
dtype: float64

In [159]:
price_dict = {'A6':20000, 'A8':12000, '3series':220000, '5series':60000, '7series':14000, 'XJ':6600, 'ES':135000, 'LS':30000, 'RX':120000, 'Sclass':25000}


price = pd.Series(price_dict,index = names)
final_rank_price = pd.concat([ranks,price],axis =1 )
final_rank_price.columns = ['rank','Sales']


In [161]:
frp = final_rank_price
print frp['rank'].corr(frp['Sales'],method='spearman')

0.721212121212
